# Combining statistics

At it's core, pavey is a set of routines to combine statistics with arbitrary weights.  This is accomplished through objects of `RunningStats...`.  There are three varieties of such objects:

 * `RunningStats` is for scalar averages/variances
 
 * `RunningStatsVec` is for arrays of averages/variances
 
 * `RunningStatsVecCov` is for arrays of averages/covariances
 


# Scalar data

In [11]:
from __future__ import print_function
import pavey
import numpy as np

Make synthetic data:

In [5]:
# set seed for consistency
np.random.seed(0)

# make collection of random samples 
min_size = 5
max_size = 100
nchunks = 10

X = []
for _ in range(nchunks):
    n = np.random.randint(min_size, max_size)
    x = np.random.rand(n)
    X.append(x)


In [8]:
# data has different weigths for each sample
print [len(x) for x in X]

[49, 17, 5, 26, 7, 57, 32, 80, 92, 51]


In [14]:
# lets check the total statistics:
Xa = np.concatenate(X)

print('shape', Xa.shape)
print('mean', Xa.mean())
print('var', Xa.var(ddof=0))

shape (416,)
mean 0.47698086027
var 0.0829144085845


But what if instead of the raw data, we only had access to the individual statistics for each chunck

In [20]:
c_mean = [x.mean() for x in X]
c_var = [x.var(ddof=0) for x in X]
c_weight = [x.size for x in X]

How do we combine these?

That's where pavey comes it!

## data from a stream

lets assume that the data comes in one at a time from another process.  How doe we combine?

In [31]:
rs = pavey.RunningStats()

In [32]:
help(rs.push_stat)

Help on method push_stat in module pavey:

push_stat(self, w, a, v) method of pavey.RunningStats instance
    accumulate statistics
    
    Parameters
    ----------
    w : int or array broadcastable to a.shape
        weight of sample
    
    a : array shape == self.shape
        averages
    v : array shape == self.var_shape



In [33]:
for (m, v, w) in zip(c_mean, c_var, c_weight):
    rs.push_stat(w=w,a=m, v=v)

In [40]:
print('total weight:', rs.weight())
print('total mean:', rs.mean())
print('total var:', rs.var())

np.testing.assert_allclose(rs.weight(), Xa.shape)
np.testing.assert_allclose(rs.mean(), Xa.mean())
np.testing.assert_allclose(rs.var(), Xa.var())

total weight: 416.0
total mean: 0.47698086027
total var: 0.0829144085845


These are exactly the same as before!

## combining in one go

Often, we have data with arbitrary weights that we want to combine in one go.  `pavey` does this as well.

In [45]:
reload(pavey)

<module 'pavey' from '/Users/wpk/Documents/python/pavey/__init__.py'>

In [48]:
rs = pavey.RunningStats()

rs.push_stats(w=c_weight, a=c_mean, v=c_var)

print('total weight:', rs.weight())
print('total mean:', rs.mean())
print('total var:', rs.var())

np.testing.assert_allclose(rs.weight(), Xa.shape)
np.testing.assert_allclose(rs.mean(), Xa.mean())
np.testing.assert_allclose(rs.var(), Xa.var())

total weight: 416.0
total mean: 0.47698086027
total var: 0.0829144085845


Again, everything lines up

# Vector data

pavey works with arbitrary shaped arrays of stats as well!

In [49]:
import random

def build_random_sample(X,samp_axis,min_chunk=4,max_chunk=20,shuffle=True):

    nsamp = X.shape[samp_axis]
    
    idx = np.arange(nsamp)
    if shuffle:
        np.random.shuffle(idx)
    
    
        
    i0=0
    i1=np.random.randint(min_chunk,max_chunk)
    XA = []
    while True:
        XA.append(np.take(X,idx[i0:i1],axis=samp_axis))
        if i1==nsamp:
            break
        i0 = i1
        i1 += np.random.randint(min_chunk,max_chunk)
        i1 = min(nsamp,i1)
        
    return XA
    

In [62]:
# random data
Xa = np.random.rand(1000, 4, 3)

# random chunked data
X = build_random_sample(Xa, samp_axis=0)



In [63]:
# chunk sizes
print([x.shape[0] for x in X])

[15, 7, 8, 6, 19, 12, 19, 14, 14, 6, 17, 4, 13, 6, 5, 15, 9, 6, 9, 16, 13, 17, 9, 15, 18, 18, 15, 13, 16, 8, 12, 13, 6, 19, 19, 14, 8, 18, 7, 17, 5, 14, 9, 12, 13, 18, 12, 17, 18, 6, 11, 18, 8, 7, 13, 15, 4, 9, 5, 5, 14, 18, 17, 16, 12, 10, 7, 18, 19, 6, 10, 6, 14, 6, 19, 12, 12, 10, 12, 15, 7, 8, 10, 8]


In [78]:
print('mean')
print(Xa.mean(axis=0))
print('var')
print(Xa.var(axis=0,ddof=0))
print('shape', Xa.shape)


mean
[[ 0.49408035  0.48503188  0.49670518]
 [ 0.49295237  0.49641412  0.51238468]
 [ 0.48979702  0.5091747   0.50774389]
 [ 0.47583748  0.51021223  0.50774218]]
var
[[ 0.08825941  0.08361924  0.07953972]
 [ 0.08204253  0.08275827  0.08462381]
 [ 0.08397126  0.08090412  0.08093979]
 [ 0.0821092   0.08374404  0.08350601]]
shape (1000, 4, 3)


### chunked data means

In [71]:
c_mean = [x.mean(axis=0) for x in X]
c_var = [x.var(axis=0,ddof=0) for x in X]
c_weight = [x.shape[0] for x in X]

In [72]:
print(c_mean[0:2])

[array([[ 0.50178533,  0.38311372,  0.49453173],
       [ 0.45362144,  0.46688966,  0.47385183],
       [ 0.54882738,  0.44084563,  0.42191511],
       [ 0.47496761,  0.55322164,  0.45717714]]), array([[ 0.33130108,  0.3481571 ,  0.47438233],
       [ 0.44898375,  0.34353588,  0.50111139],
       [ 0.61900463,  0.46750933,  0.5868955 ],
       [ 0.43118281,  0.60157466,  0.43293204]])]


Lets combine our data in stream

In [74]:
rs = pavey.RunningStatsVec(shape=(4,3))

In [75]:
for (m, v, w) in zip(c_mean, c_var, c_weight):
    rs.push_stat(w=w,a=m,v=v)

In [84]:
print('total mean')
print(rs.mean())

print('total var')
print(rs.var())

print('total weight')
print(rs.weight())

np.testing.assert_allclose(rs.weight(), Xa.shape[0])
np.testing.assert_allclose(rs.mean(), Xa.mean(axis=0))
np.testing.assert_allclose(rs.var(), Xa.var(axis=0))

total mean
[[ 0.49408035  0.48503188  0.49670518]
 [ 0.49295237  0.49641412  0.51238468]
 [ 0.48979702  0.5091747   0.50774389]
 [ 0.47583748  0.51021223  0.50774218]]
total var
[[ 0.08825941  0.08361924  0.07953972]
 [ 0.08204253  0.08275827  0.08462381]
 [ 0.08397126  0.08090412  0.08093979]
 [ 0.0821092   0.08374404  0.08350601]]
total weight
[[ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]]


* All good!
* Note that weights have been broadcast to the shape of mean/var.  Different wieghts can apply to different dimensions if desired

### one go

In [86]:
rs = pavey.RunningStatsVec(shape=(4,3))
rs.push_stats(w=c_weight, a=c_mean, v=c_var)

In [87]:
print('total mean')
print(rs.mean())

print('total var')
print(rs.var())

print('total weight')
print(rs.weight())

np.testing.assert_allclose(rs.weight(), Xa.shape[0])
np.testing.assert_allclose(rs.mean(), Xa.mean(axis=0))
np.testing.assert_allclose(rs.var(), Xa.var(axis=0))

total mean
[[ 0.49408035  0.48503188  0.49670518]
 [ 0.49295237  0.49641412  0.51238468]
 [ 0.48979702  0.5091747   0.50774389]
 [ 0.47583748  0.51021223  0.50774218]]
total var
[[ 0.08825941  0.08361924  0.07953972]
 [ 0.08204253  0.08275827  0.08462381]
 [ 0.08397126  0.08090412  0.08093979]
 [ 0.0821092   0.08374404  0.08350601]]
total weight
[[ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]]


## covariance

Note that `numpy.cov` only handles a single variable dimension.  pavey includes a routine to handle arbitrary dimensions.

In [91]:
cov = pavey.cov_nd(Xa, axis=0)
print(cov.shape)

(4, 3, 4, 3)


In [92]:
c_cov = [pavey.cov_nd(x, axis=0) for x in X]

### stream

In [99]:
rs = pavey.RunningStatsVecCov(shape=(4,3))

for (w, m, c) in zip(c_weight, c_mean, c_cov):
    rs.push_stat(w=w, a=m, v=c)

In [100]:
print('total mean')
print(rs.mean())

print('total var')
print(rs.var())

print('total weight')
print(rs.weight())

np.testing.assert_allclose(rs.weight(), Xa.shape[0])
np.testing.assert_allclose(rs.mean(), Xa.mean(axis=0))
np.testing.assert_allclose(rs.var(), Xa.var(axis=0))

# final test is on cov
np.testing.assert_allclose(rs.cov(), cov)

total mean
[[ 0.49408035  0.48503188  0.49670518]
 [ 0.49295237  0.49641412  0.51238468]
 [ 0.48979702  0.5091747   0.50774389]
 [ 0.47583748  0.51021223  0.50774218]]
total var
[[ 0.08825941  0.08361924  0.07953972]
 [ 0.08204253  0.08275827  0.08462381]
 [ 0.08397126  0.08090412  0.08093979]
 [ 0.0821092   0.08374404  0.08350601]]
total weight
[[ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]]


### one go

In [101]:
rs = pavey.RunningStatsVecCov(shape=(4,3))
rs.push_stats(w=c_weight, a=c_mean, v=c_cov)

In [102]:
print('total mean')
print(rs.mean())

print('total var')
print(rs.var())

print('total weight')
print(rs.weight())

np.testing.assert_allclose(rs.weight(), Xa.shape[0])
np.testing.assert_allclose(rs.mean(), Xa.mean(axis=0))
np.testing.assert_allclose(rs.var(), Xa.var(axis=0))

# final test is on cov
np.testing.assert_allclose(rs.cov(), cov)

total mean
[[ 0.49408035  0.48503188  0.49670518]
 [ 0.49295237  0.49641412  0.51238468]
 [ 0.48979702  0.5091747   0.50774389]
 [ 0.47583748  0.51021223  0.50774218]]
total var
[[ 0.08825941  0.08361924  0.07953972]
 [ 0.08204253  0.08275827  0.08462381]
 [ 0.08397126  0.08090412  0.08093979]
 [ 0.0821092   0.08374404  0.08350601]]
total weight
[[ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]]


# Block averages

* Suppose the synthetic data above is from a simulation.
* We'd like to perform block averages on the data.
* That is, combine the data from sampel 0 and 1, 2 and 3, ....
* repeat


In [115]:
blocker = pavey.RunningStatsList.from_stats(w=np.array(c_weight), a=np.array(c_mean), var=np.array(c_var), axis=0)

In [116]:
blocker.shape

(84,)

In [139]:
# combine into single sample
out = blocker.combine(block_size=None)

rs = out[0]

print('total mean')
print(rs.mean())

print('total var')
print(rs.var())

print('total weight')
print(rs.weight())

np.testing.assert_allclose(rs.weight(), Xa.shape[0])
np.testing.assert_allclose(rs.mean(), Xa.mean(axis=0))
np.testing.assert_allclose(rs.var(), Xa.var(axis=0))

# final test is on cov
#np.testing.assert_allclose(rs.cov(), cov)

total mean
[[ 0.49408035  0.48503188  0.49670518]
 [ 0.49295237  0.49641412  0.51238468]
 [ 0.48979702  0.5091747   0.50774389]
 [ 0.47583748  0.51021223  0.50774218]]
total var
[[ 0.08825941  0.08361924  0.07953972]
 [ 0.08204253  0.08275827  0.08462381]
 [ 0.08397126  0.08090412  0.08093979]
 [ 0.0821092   0.08374404  0.08350601]]
total weight
[[ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]
 [ 1000.  1000.  1000.]]


In [140]:
## Standard errors of statistics

In [141]:
print(blocker.mean_SEM())

[[ 0.00929754  0.00890924  0.00873288]
 [ 0.00891848  0.01008172  0.01040513]
 [ 0.0091682   0.00903575  0.0093628 ]
 [ 0.00770296  0.00930417  0.00851316]]


In [142]:
print(blocker.var_SEM())

[[ 0.00278505  0.00221365  0.00236944]
 [ 0.00231612  0.00240477  0.00240101]
 [ 0.00228664  0.00260172  0.00235317]
 [ 0.00229372  0.00207687  0.00290443]]


### combine nearest two samples

In [149]:
blocker2 = blocker.combine(block_size=2)

In [153]:
blocker2.shape

(42,)

In [154]:
print(blocker2.mean_SEM())

[[ 0.0081124   0.00858759  0.00892054]
 [ 0.01009812  0.01070748  0.01042609]
 [ 0.00923334  0.00987257  0.00911865]
 [ 0.00777031  0.00916571  0.00848954]]


In [156]:
## again

blocker4a = blocker.combine(block_size=4)
blocker4b = blocker2.combine(block_size=2)

In [158]:
print(blocker4a.shape)
print(blocker4b.shape)

(21,)
(21,)


In [160]:
print(blocker4a.mean_SEM())

[[ 0.00797114  0.01003843  0.00991918]
 [ 0.01124391  0.01129885  0.00774526]
 [ 0.00959625  0.00579277  0.01017249]
 [ 0.00873978  0.00991927  0.00874668]]


In [161]:
print(blocker4b.mean_SEM())

[[ 0.00797114  0.01003843  0.00991918]
 [ 0.01124391  0.01129885  0.00774526]
 [ 0.00959625  0.00579277  0.01017249]
 [ 0.00873978  0.00991927  0.00874668]]
